In [ ]:
tarekziade/profanity-clean

In [ ]:
!pip install -U transformers datasets huggingface_hub tensorboard==2.11

In [13]:
import torch
from datasets import load_dataset, Dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
)
import os
from huggingface_hub import HfFolder, login

In [2]:
from google.colab import userdata
login(token= userdata.get("HF_TOKEN"))

In [8]:
ds = load_dataset("tarekziade/profanity-clean")

README.md:   0%|          | 0.00/337 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/5.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/91222 [00:00<?, ? examples/s]

In [4]:
from fthelperutils import *

In [6]:
lgobj = intializelog()

In [ ]:
lgobj.info(f"Dataset size: {len(ds['train'])}")

INFO:root:Dataset size: 91222


In [9]:
dssplit = create_ds_splits(logger=lgobj,ds=ds,testsplitsize=0.4, seed=42, testvalidsplit=0.5)

INFO:root:Training dataset size: 54733
INFO:root:Validation dataset size: 18244
INFO:root:Test dataset size: 18245


In [ ]:
print( len(dssplit['train']))

54733


In [ ]:
dssplit['validation'][:10]

{'is_offensive': [0, 0, 0, 0, 1, 1, 1, 0, 0, 1],
 'text': ['If we can t syop editing out others comments then lets just keep ALL versions',
  'Please add and clean up the page',
  'Watch your edit summaries diff Please do not attack other editors If you continue you will be blocked from editing Wikipedia',
  'Sorry I have got two cats from Stirling',
  'Turn a hoe into a house wife you bet not',
  'Why don t you fuck off GET SOME JOBE TO DO IDIOT May',
  'Not the type to take it slow turn a good girl into a hoe spookyblack',
  'The Gathering page From the Vault Dragons',
  'February UTC You can post it yourself my point was just that you should stick to discussing the article at hand nothing else That s all khoi',
  'People like you are enemy of the peace think wider be an honest man not unhappy man cheers']}

In [10]:
trainset = dssplit['train']
valset = dssplit['validation']
testset = dssplit['test']

In [11]:
from transformers import RobertaTokenizer, TFRobertaModel

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = TFRobertaModel.from_pretrained('roberta-base')
# apply padding and truncation to ensure that all text have the same length (256 tokens).
def apply_tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=256)

train_ds = trainset.map(apply_tokenize, batched=True, batch_size=len(trainset))
val_ds = valset.map(apply_tokenize, batched=True, batch_size=len(valset))
test_ds = testset.map(apply_tokenize, batched=True, batch_size=len(testset))

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['roberta.embeddings.position_ids', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

Map:   0%|          | 0/54733 [00:00<?, ? examples/s]

Map:   0%|          | 0/18244 [00:00<?, ? examples/s]

Map:   0%|          | 0/18245 [00:00<?, ? examples/s]

In [16]:
train_dataset = train_ds.with_format("tf")
valid_dataset = val_ds.with_format("tf")
test_dataset = test_ds.with_format("tf")

In [20]:
def apply_labels(example):
    example['labels'] = example['is_offensive']
    return example

train_ds = train_ds.map(apply_labels)
val_ds = val_ds.map(apply_labels)
test_ds = test_ds.map(apply_labels)

Map:   0%|          | 0/54733 [00:00<?, ? examples/s]

Map:   0%|          | 0/18244 [00:00<?, ? examples/s]

Map:   0%|          | 0/18245 [00:00<?, ? examples/s]

In [22]:
model_id = "roberta-base"
config = AutoConfig.from_pretrained(model_id)

# Default considers AdamW optimizer

In [23]:
model = RobertaForSequenceClassification.from_pretrained(model_id, config=config)

# TrainingArguments
training_args = TrainingArguments(
    output_dir='/content/resultmdl',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=500,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="tensorboard",
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [24]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.339000,0.246355
2,0.639000,0.632718
3,0.629800,0.631900
4,0.639100,0.631898
5,0.641500,0.631924


TrainOutput(global_step=34210, training_loss=0.5511772137920399, metrics={'train_runtime': 1610.4134, 'train_samples_per_second': 169.935, 'train_steps_per_second': 21.243, 'total_flos': 8719269124679400.0, 'train_loss': 0.5511772137920399, 'epoch': 5.0})

In [25]:
trainer.evaluate()

{'eval_loss': 0.2463546246290207,
 'eval_runtime': 27.0424,
 'eval_samples_per_second': 674.645,
 'eval_steps_per_second': 84.349,
 'epoch': 5.0}

In [32]:
trainedmdlpath = "/content/resultmdl/checkpoint-6842"
from transformers import AutoModelForSequenceClassification
trainedmdl = AutoModelForSequenceClassification.from_pretrained(trainedmdlpath)

In [41]:
y_true = test_ds['is_offensive']
y_pred = trainer.predict(test_ds)


# converts the raw observed probabilities to class lables (0 or 1) by picking the class with high probability for every sample


In [43]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

#converts (probabilities) into hard predictions (class labels) by picking the class with highest predicted probability per sample
y_predlbl = y_pred.predictions.argmax(axis=1)

Accuracy = accuracy_score(y_true, y_predlbl)
Precision = precision_score(y_true, y_predlbl)
Recall = recall_score(y_true, y_predlbl)
F1 = f1_score(y_true, y_predlbl)

print(f"Observed Accuracy: {Accuracy}")
print(f"Observed Precision: {Precision}")
print(f"Observed Recall: {Recall}")
print(f"Observed F1-score: {F1}")

Observed Accuracy: 0.9431625102767882
Observed Precision: 0.9094307561597281
Observed Recall: 0.9139344262295082
Observed F1-score: 0.911677029213866


In [47]:
# save tokenizer
tokenizer.save_pretrained('/content/resultmdl/')

('/content/resultmdl/tokenizer_config.json',
 '/content/resultmdl/special_tokens_map.json',
 '/content/resultmdl/vocab.json',
 '/content/resultmdl/merges.txt',
 '/content/resultmdl/added_tokens.json')

In [55]:
# read tokenizer

tokenizer = AutoTokenizer.from_pretrained("/content/resultmdl/")

def testmdl(textstr, trainedmdl, tokenizer):
    import torch
    sentence = textstr
    inputs = tokenizer(sentence, return_tensors="pt")
    with torch.no_grad():
        outputs = trainedmdl(**inputs)
        logits = outputs.logits
        predictions = logits.argmax(-1).item()

    return predictions

In [60]:
result = testmdl(textstr="You asshole", trainedmdl=trainedmdl, tokenizer=tokenizer)

In [58]:
result

1

In [59]:
class_names = ["notoffensive", "offensive"]
predicted_label = class_names[result]
print(f"Predicted class: {predicted_label}")

Predicted class: offensive


In [61]:
result = testmdl(textstr="You are a good person", trainedmdl=trainedmdl, tokenizer=tokenizer)

In [65]:
result

0

In [62]:
class_names = ["notoffensive", "offensive"]
predicted_label = class_names[result]
print(f"Predicted class: {predicted_label}")

Predicted class: notoffensive
